In [ ]:
from db import DB
from utils.os import load_json
from models.pytorch.us_gaap_alignment.ingest import insert_ofss_data


db = DB()

# Path to OFSS schema file
schema_file = '../shared/open_financial_statement_schema.json'

# Load the schema from the JSON file
schema_data = load_json(schema_file)

# Insert the data into the database
insert_ofss_data(db, schema_data)